In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "simulated_annealing"
    cd(joinpath(@__DIR__, "../../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMol.Algorithms: SimulatedAnnealing, simulate!
using MorphoMolNotebooks: configuration_to_poly, calculate_initial_temperature, add_to_simulation_minima!, has_hard_sphere_overlap, is_dispersed, get_random_configuration_without_overlap_in_bounds, is_configuration_equivalent_to_mcc, is_equivalent_configuration, get_mccs, minimal_mcc, get_energies_of_mccs
using GLMakie
using Random
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
# known_minimal_configurations = Dict{@NamedTuple{n::Int64, rs::Float64, eta::Float64}, @NamedTuple{centers::Vector{Float64}, E::Float64, mcc::Bool, dispersed::Bool}}()
# @save "src/assets/virial_known_minimal_configurations.jld2" known_minimal_configurations

In [2]:
pf_id = "virial"
@load "src/assets/$(pf_id)_known_minimal_configurations.jld2" known_minimal_configurations
scan_dir = "assets/parameter_space_scans/$(pf_id)/"
for file in filter(x -> occursin(".jld2", x), readdir(scan_dir))
    @load "$(scan_dir)$file" lowest_energy_simulated_states lowest_energy_no_mcc_simulated_states
    for (k, v) in lowest_energy_simulated_states
        if has_hard_sphere_overlap(v.centers)
            println("Hard sphere overlap in $(file)")
            continue
        end
        
        mcc = is_configuration_equivalent_to_mcc(v.centers; threshold=1.0 + k.rs / 2.0)
        dispersed = is_dispersed(v.centers, k.rs)
        if !haskey(known_minimal_configurations, k)
            known_minimal_configurations[k] = (centers = v.centers, E = v.E, mcc = mcc, dispersed = dispersed)
        elseif known_minimal_configurations[k].E >= v.E
            known_minimal_configurations[k] = (centers = v.centers, E = v.E, mcc = mcc, dispersed = dispersed)
        end
    end
end
@save "src/assets/$(pf_id)_known_minimal_configurations.jld2" known_minimal_configurations

In [51]:
# @load "src/assets/$(pf_id)_known_minimal_configurations.jld2" known_minimal_configurations
# for (k, v) in known_minimal_configurations
#     println(k, " ", v)
# end

In [3]:
n_to_check = 8
try
    rm("assets/output", recursive = true)
catch 
end
mkpath("assets/output")
i = 1
for (k, v) in known_minimal_configurations
    if k.n != n_to_check #|| k.rs < 1.0
        continue
    end
    n = k.n
    rs = k.rs
    eta = k.eta
    pf = MorphoMol.Energies.get_virial_prefactors(rs, eta)
    E_mcc = minimal_mcc(n, rs, pf).E
    mcc_name = minimal_mcc(n, rs, pf).name
    E_sim = MorphoMol.Energies.solvation_free_energy(v.centers, 1, [1.0 for _ in 1:n], rs, pf, 100000.0, 100000.0, 100.0)
    print("$i rs: $rs eta: $eta | MCC: $(E_mcc), Sim: $(E_sim)")
    if E_mcc > E_sim && !(occursin("dispersed", mcc_name) && isapprox(E_mcc, E_sim))
        println(" | SUCCESS: Minimum is more stable than MCC")
        mcc = is_configuration_equivalent_to_mcc(v.centers; threshold=1.0 + rs/2.0)
        configuration_to_poly(v.centers, [1.0 for _ in 1:k.n], "assets/output/$(i)", k.n, 1)
        i += 1
    elseif occursin("dispersed", mcc_name) && isapprox(E_mcc, E_sim)
        println("| DISPERSED")
    else
        println()
    end
end

1 rs: 0.8 eta: 0.515 | MCC: 210.83755999863382, Sim: 213.8627466077205
1 rs: 0.525 eta: 0.565 | MCC: 951.924610817006, Sim: 980.9333287625834
1 rs: 0.7 eta: 0.59 | MCC: 545.0438691630245, Sim: 496.53048887270154 | SUCCESS: Minimum is more stable than MCC
2 rs: 0.875 eta: 0.565 | MCC: 247.813289943993, Sim: 226.00969112615215 | SUCCESS: Minimum is more stable than MCC
3 rs: 0.95 eta: 0.49 | MCC: 113.79900495100146, Sim: 115.7419266358362
3 rs: 0.6 eta: 0.54 | MCC: 541.8994232649904, Sim: 558.9521208722421
3 rs: 0.475 eta: 0.64 | MCC: 2453.1353674571187, Sim: 2458.0582791240427
3 rs: 0.8 eta: 0.54 | MCC: 255.72442171989172, Sim: 242.69178881932504 | SUCCESS: Minimum is more stable than MCC
4 rs: 1.025 eta: 0.49 | MCC: 94.48082850934995, Sim: 97.15652642411241
4 rs: 0.975 eta: 0.565 | MCC: 188.25163699991822, Sim: 156.41177394024777 | SUCCESS: Minimum is more stable than MCC
5 rs: 0.375 eta: 0.5 | MCC: 1394.2820582594472, Sim: 1420.2144158156193
5 rs: 0.9 eta: 0.565 | MCC: 230.60894329724